In [ ]:
import pandas as pd
import scipy
import networkx as nx
import collections
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ipysigma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.6 MB/s eta 0:00:00


In [ ]:

from ipysigma import Sigma


In [ ]:
directory = '/content/drive/MyDrive/Colab Notebooks/SNA2 Project/socialmediadata-tweets-of-congress-august'

# JSON data storage
json_data = []

# directory loop
for filename in os.listdir(directory):

    if filename.endswith('.json'):
        filepath = os.path.join(directory, filename)

        # Read json data and append to list
        with open(filepath, 'r') as file:
            json_data.append(pd.read_json(file))


# combine all dataframes
combined_df = pd.concat(json_data, ignore_index=True)

# Display the combined DataFrame
print(combined_df)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/SNA2 Project/socialmediadata-tweets-of-congress-august'

In [ ]:
cols = ["id", "screen_name", "text"]

tweets_df = combined_df[cols].copy()

tweets_df

,id,screen_name,text
0,893320874052419584,auctnr1,Hardware store chain Sutherlands opens new Spr...
1,893335640548364288,RosLehtinen,RT @MichaelSteele It's an honor to be a part o...
2,893334335083782144,RosLehtinen,RT @SCClemons Am here at Gold Coast 25th Anniv...
3,893333304933535744,keithellison,Decline In Democracy Spreads Across The Globe ...
4,893333281541951488,keithellison,Decline In Democracy Spreads Across The Globe ...
...,...,...,...
45631,899109294171451392,LacyClayMO1,A #STL original who broke barriers with his ta...
45632,899108165131829248,LacyClayMO1,"Comic genius, fearless fighter for social just..."
45633,899107815494701056,RepYvetteClarke,"We will never forget #DickGregory, a genuine t..."
45634,899107299649835008,BarbaraComstock,When out on the road in #VA10 like to stop by ...


In [ ]:
#Finding party affiliation

congress_party_csv = '/content/drive/MyDrive/Colab Notebooks/SNA2 Project/socialmediadata-tweets-of-congress-august/retweets_congress.csv'


party_df = pd.read_csv(congress_party_csv)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/SNA2 Project/socialmediadata-tweets-of-congress-august/retweets_congress.csv'

In [ ]:
party_dict = dict(zip(party_df['user_screen_name'], party_df['party']))


tweets_df['screen_name'] = tweets_df['screen_name'].str.lower()

# def find_party(person):
#   return party_df.loc[party_df['user_screen_name'] == person , 'party']

# Add a new column to combined_df with the party affiliation based on tweeter's name
tweets_df['party'] = tweets_df['screen_name'].map(party_dict)
tweets_df['party'] = tweets_df['party'].fillna('NaN').astype(str)

# for i in tweets_df.itertuples():
#     person = i[2]
#     i[5] = find_party(person)

NameError: name 'party_df' is not defined

In [ ]:
#function to extract tweets

def references(tweet):
  return [i for i in tweet.split() if i.startswith("@")]

tweets_df['references'] = tweets_df['text'].apply(references)



In [ ]:
tweets_df

#print(type(tweets_df["party"][0]))

,id,screen_name,text,party,references
0,893320874052419584,auctnr1,Hardware store chain Sutherlands opens new Spr...,NaN,[@springfieldNL]
1,893335640548364288,roslehtinen,RT @MichaelSteele It's an honor to be a part o...,Republican,"[@MichaelSteele, @RosLehtinen, @SCClemons]"
2,893334335083782144,roslehtinen,RT @SCClemons Am here at Gold Coast 25th Anniv...,Republican,"[@SCClemons, @RosLehtinen]"
3,893333304933535744,keithellison,Decline In Democracy Spreads Across The Globe ...,Democrat,[]
4,893333281541951488,keithellison,Decline In Democracy Spreads Across The Globe ...,Democrat,[]
...,...,...,...,...,...
45631,899109294171451392,lacyclaymo1,A #STL original who broke barriers with his ta...,NaN,[]
45632,899108165131829248,lacyclaymo1,"Comic genius, fearless fighter for social just...",NaN,[]
45633,899107815494701056,repyvetteclarke,"We will never forget #DickGregory, a genuine t...",Democrat,[]
45634,899107299649835008,barbaracomstock,When out on the road in #VA10 like to stop by ...,NaN,[]


In [ ]:
tweets_df.replace("NaN", np.nan, inplace=True)

# Iterating through DataFrame and removing rows with NaN values
for index, row in tweets_df.iterrows():
    if row.isnull().any():
        tweets_df.drop(index, inplace=True)

In [ ]:
tweets_df

,id,screen_name,text,party,references
1,893335640548364288,roslehtinen,RT @MichaelSteele It's an honor to be a part o...,Republican,"[@MichaelSteele, @RosLehtinen, @SCClemons]"
2,893334335083782144,roslehtinen,RT @SCClemons Am here at Gold Coast 25th Anniv...,Republican,"[@SCClemons, @RosLehtinen]"
3,893333304933535744,keithellison,Decline In Democracy Spreads Across The Globe ...,Democrat,[]
4,893333281541951488,keithellison,Decline In Democracy Spreads Across The Globe ...,Democrat,[]
5,893331490259578880,reprichhudson,RT @DavidWhisenant The closing of Innes St. Dr...,Republican,"[@DavidWhisenant, @RepRichHudson, @DrinkCheerw..."
...,...,...,...,...,...
45622,899117691633618944,keithellison,"Dick Gregory, may God Bless you and Keep you. ...",Democrat,[]
45623,899117107039789056,repgwenmoore,"#DickGregory's humor, insight, &amp; activism ...",Democrat,[]
45627,899113243305680896,repdonbacon,RT @RalstonPolice Thinking of our brothers and...,Republican,[@RalstonPolice]
45630,899110911507562496,repyvetteclarke,RT @amjoyshow #PMJoy--&gt;Tens of thousands ma...,Democrat,[@amjoyshow]


In [ ]:
tweets_df.to_csv('tweets.csv', index=False)

In [ ]:
references_df = pd.DataFrame(columns=['tweeter', 'relationships'])

tweet_dict = {}

#god my code is so fucking inefficient
for index, row in tweets_df.iterrows():
    source = "@" + row['screen_name']

    reference = row['references']
    if reference != []:
      if source not in tweet_dict.keys():
        source=source[1:]
        tweet_dict[source] = []
        for ref in reference:
          tweet_dict[source].append(ref[1:].lower())
      else:
        for ref in reference:
          tweet_dict[source].append(ref)

color_dict= {}
party_dict = {'Democrat': 'blue', 'Republican': 'red', "NaN": "gray"}

#print(tweets_df.loc[tweets_df['screen_name'] == 'replancenj7', 'party'])
for index, row in tweets_df.iterrows():
  source = row["screen_name"]
  if source not in color_dict:
    color_dict[source] = party_dict.get(tweets_df.loc[tweets_df['screen_name'] == source, 'party'].iloc[0])



In [ ]:
color_dict

{'roslehtinen': 'red',
 'keithellison': 'blue',
 'reprichhudson': 'red',
 'repmialove': 'red',
 'warrendavidson': 'red',
 'replipinski': 'blue',
 'repkaygranger': 'red',
 'reploudermilk': 'red',
 'johncornyn': 'red',
 'repsinema': 'blue',
 'repjoecrowley': 'blue',
 'replobiondo': 'red',
 'repwebster': 'red',
 'royblunt': 'red',
 'repdennisross': 'red',
 'senschumer': 'blue',
 'repbillfoster': 'blue',
 'repbilljohnson': 'red',
 'repsmucker': 'red',
 'repleezeldin': 'red',
 'senwhitehouse': 'blue',
 'judgecarter': 'red',
 'repgoodlatte': 'red',
 'reppaulmitchell': 'red',
 'congressmanraja': 'blue',
 'repkevinbrady': 'red',
 'speakerryan': 'red',
 'repbrianhiggins': 'blue',
 'repbyrne': 'red',
 'randpaul': 'red',
 'replloyddoggett': 'blue',
 'repdianeblack': 'red',
 'repgrothman': 'red',
 'sendonnelly': 'blue',
 'senjohnmccain': 'red',
 'repcomstock': 'red',
 'repshimkus': 'red',
 'repcharliedent': 'red',
 'stevedaines': 'red',
 'repseanduffy': 'red',
 'repbillflores': 'red',
 'reprwillia

Double-checked the data; there are only 2 independents in Congress at the time:

seanaugustking
berniesanders || sensanders

In [ ]:

#create edgelist

edges = []

for key in tweet_dict:
    ppl = tweet_dict[key]
    for person in ppl:
        edges.append((key, person))


print(edges)

#create weighted graph
G = nx.Graph()

for edge in edges:
    if G.has_edge(edge[0], edge[1]):
        #increase weight
        G[edge[0]][edge[1]]['weight'] += 1
    else:
        #initialize weight
        G.add_edge(edge[0], edge[1], weight=1)


[('roslehtinen', 'statedept'), ('roslehtinen', 'fiu'), ('roslehtinen', 'mdcollege,'), ('reprichhudson', 'nascar'), ('repmialove', "doterra's"), ('johncornyn', 'byronyork'), ('repsinema', 'hurdonthehill!'), ('repjoecrowley', 'abcpolitics'), ('replobiondo', '1400wond.'), ('replobiondo', 'maharshipatel'), ('replobiondo', 'replobiondo'), ('repwebster', 'fpeahomeschool.'), ('repdennisross', 'natlparkservice!'), ('repdennisross', 'natlparkservice'), ('senschumer', 'medium.'), ('judgecarter', 'ups'), ('reppaulmitchell', 'katebolduan'), ('reppaulmitchell', 'cnn'), ('speakerryan', 'housegop'), ('repbrianhiggins', 'thebuffalonews'), ('repbrianhiggins', 'bflonavalpark'), ('repbyrne', 'nbcpolitics'), ('replloyddoggett', 'dvisd_athletics'), ('replloyddoggett', 'replloyddoggett'), ('repgrothman', 'fox11news'), ('repgrothman', 'repgrothman'), ('randpaul', 'rsi'), ('randpaul', 'steena12'), ('randpaul', 'dcexaminer'), ('randpaul', 'randpaul'), ('randpaul', 'kamalaharris'), ('sendonnelly', 'usdafsa'), (

In [ ]:
G.degree()

DegreeView({'roslehtinen': 3, 'statedept': 1, 'fiu': 1, 'mdcollege,': 1, 'reprichhudson': 1, 'nascar': 1, 'repmialove': 1, "doterra's": 1, 'johncornyn': 1, 'byronyork': 1, 'repsinema': 1, 'hurdonthehill!': 1, 'repjoecrowley': 1, 'abcpolitics': 1, 'replobiondo': 4, '1400wond.': 1, 'maharshipatel': 1, 'repwebster': 1, 'fpeahomeschool.': 1, 'repdennisross': 2, 'natlparkservice!': 1, 'natlparkservice': 2, 'senschumer': 1, 'medium.': 1, 'judgecarter': 1, 'ups': 1, 'reppaulmitchell': 2, 'katebolduan': 1, 'cnn': 3, 'speakerryan': 3, 'housegop': 2, 'repbrianhiggins': 2, 'thebuffalonews': 1, 'bflonavalpark': 1, 'repbyrne': 1, 'nbcpolitics': 1, 'replloyddoggett': 3, 'dvisd_athletics': 1, 'repgrothman': 3, 'fox11news': 1, 'randpaul': 6, 'rsi': 1, 'steena12': 1, 'dcexaminer': 1, 'kamalaharris': 1, 'sendonnelly': 3, 'usdafsa': 1, 'indystatefair': 1, 'usda': 1, 'senjohnmccain': 5, 'joelieberman': 2, 'meghanmccain': 2, 'lindseygrahamsc': 3, 'repcharliedent': 5, 'mowlehighcounty': 1, 'mealsonwheelspa'

In [ ]:
#Degree centrality

degrees = G.degree()

Sigma(G, node_size = degrees, node_size_scale = ("log"), node_size_range = (0.1, 10), node_color=color_dict)


NameError: name 'G' is not defined

In [ ]:
#Eigenvector centrality

referees =nx.eigenvector_centrality(G,max_iter=200)

Sigma(G, node_size = referees, node_size_scale = ("sqrt"), node_size_range = (1, 10), node_color=color_dict)

NameError: name 'G' is not defined

In [ ]:
border_phrases = ["border", "immigrant", "immigration", "mexico", "wall", "black", "african", "crime", "rape", "drug", "rapist"]

border_tweet_df = tweets_df[tweets_df['text'].str.contains('|'.join(border_phrases), case=False)]

In [ ]:
border_tweet_df

,id,screen_name,text,party,references
5,893331490259578880,reprichhudson,RT @DavidWhisenant The closing of Innes St. Dr...,Republican,"[@DavidWhisenant, @RepRichHudson, @DrinkCheerw..."
76,893455129441169408,senschumer,"President Trump wants to cut immigration, exce...",Democrat,[@syracusedotcom:]
129,893445862252326912,johncornyn,RT @MattMackowiak Smart border security plan f...,Republican,"[@MattMackowiak, @JohnCornyn]"
177,893468341712674816,call_me_dutch,Joined @SenatorCardin @ChrisVanHollen @RepSarb...,Democrat,"[@SenatorCardin, @ChrisVanHollen, @RepSarbanes..."
206,893463023138033664,repespaillat,Still much work remains in our efforts to crea...,Democrat,[@USATODAYmoney]
...,...,...,...,...,...
45497,899015574415699968,repgaramendi,Hosting a town hall in Winters right now. Taki...,Democrat,[]
45501,899015338913705984,johncornyn,But not Justice Thomas? Smithsonian African Am...,Republican,[@dcexaminer]
45522,899037805967319040,repjayapal,Next neighborhood meetup is Thursday in Wallin...,Democrat,[]
45554,899053591977119744,sensherrodbrown,"Sebastian Gorka, a deputy assistant to the pre...",Democrat,[]


In [ ]:
border_edges = []

for key in tweet_dict:

    if key in border_tweet_df["screen_name"].values:
      ppl = tweet_dict[key]
      for person in ppl:
          border_edges.append((key, person))

In [ ]:
G_border = nx.Graph()

for b_edge in border_edges:
    if G_border.has_edge(b_edge[0], b_edge[1]):
        #increase weight
        G_border[b_edge[0]][b_edge[1]]['weight'] += 1
    else:
        #initialize weight
        G_border.add_edge(b_edge[0], b_edge[1], weight=1)


In [ ]:
b_degrees = G_border.degree()

Sigma(G_border, node_size = b_degrees, node_size_scale = ("log"), node_size_range = (0.1, 10), node_color=color_dict)


Sigma(nx.Graph with 656 nodes and 495 edges)

In [ ]:
border_referees =nx.eigenvector_centrality(G_border,max_iter=200)

Sigma(G_border, node_size = border_referees, node_size_scale = ("sqrt"), node_size_range = (1, 10), node_color=color_dict)

Sigma(nx.Graph with 656 nodes and 495 edges)

In [ ]:
health_phrases = ["drug", "healthcare", "pharma", "medic", "hospital", "doctor", "care", "illness", "insurance", "obamacare"]

health_tweet_df = tweets_df[tweets_df['text'].str.contains('|'.join(health_phrases), case=False)]

In [ ]:
health_tweet_df

,id,screen_name,text,party,references
5,893331490259578880,reprichhudson,RT @DavidWhisenant The closing of Innes St. Dr...,Republican,"[@DavidWhisenant, @RepRichHudson, @DrinkCheerw..."
9,893341646376587264,warrendavidson,"Rather than August out, Congress could focus t...",Republican,[]
19,893419976476028928,reprichhudson,"RT @mike_pence Men &amp; women of @TNGOP, make...",Republican,"[@mike_pence, @TNGOP,]"
81,893453520061566976,replipinski,Let’s Stop the Bickering and Fix the Health Ca...,Democrat,[]
82,893453514688671744,replloyddoggett,RT @fox7austin @RepLloydDoggett joins us in th...,Democrat,"[@fox7austin, @RepLloydDoggett]"
...,...,...,...,...,...
45342,898969382147080192,repbera,"On education: ""Im the product of California pu...",Democrat,[]
45375,898990451625082880,repricklarsen,Join me for two Senior Forums to help answer a...,Democrat,[]
45478,899024525047857152,reppeteaguilar,Had a great time this morning celebrating the ...,Democrat,[@CARA_Activist]
45497,899015574415699968,repgaramendi,Hosting a town hall in Winters right now. Taki...,Democrat,[]


In [ ]:
health_edges = []

for key in tweet_dict:

    if key in health_tweet_df["screen_name"].values:
      ppl = tweet_dict[key]
      for person in ppl:
          health_edges.append((key, person))


G_health = nx.Graph()

for h_edge in health_edges:
    if G_health.has_edge(h_edge[0], h_edge[1]):
        #increase weight
        G_health[h_edge[0]][h_edge[1]]['weight'] += 1
    else:
        #initialize weight
        G_health.add_edge(h_edge[0], h_edge[1], weight=1)

In [ ]:
h_degrees = G_health.degree()

Sigma(G_health, node_size = h_degrees, node_size_scale = ("log"), node_size_range = (0.1, 10), node_color=color_dict)


Sigma(nx.Graph with 779 nodes and 577 edges)

In [ ]:
health_referees =nx.eigenvector_centrality(G_health,max_iter=200)

Sigma(G_health, node_size = health_referees, node_size_scale = ("sqrt"), node_size_range = (1, 10), node_color=color_dict)

NameError: name 'G_health' is not defined

In [ ]:
military_phrases = ["serve", "national", "security", "war", "army", "navy", "threat", "NATO", "allies", "Korea", "soldier", "veteran", "vet", "air", "troop"]

military_tweet_df = tweets_df[tweets_df['text'].str.contains('|'.join(military_phrases), case=False)]

military_tweet_df

,id,screen_name,text,party,references
12,893339434526973952,roslehtinen,RT @SenatorWong Discussing the importance of c...,Republican,"[@SenatorWong, @RepBradWenstrup, @RepJoeCourtn..."
15,893408741156106240,repkaygranger,Today marks the 227th birthday of the U.S. Coa...,Republican,[]
24,893440020048224256,reprichhudson,Chris and Jonathan gave their lives protecting...,Republican,[]
25,893439973239771136,reprichhudson,We also pray for those who served alongside th...,Republican,[]
30,893438856330190848,reprichhudson,My wife Renee and I join our community in mour...,Republican,"[@FtBraggNC, @82ndABNDiv]"
...,...,...,...,...,...
45590,899086914988494848,joaquincastrotx,RT @TexasObserver USDA Rural Development helpe...,Democrat,[@TexasObserver]
45593,899084944772911104,drnealdunnfl2,"This week, I exchanged coins with Chief Master...",Republican,[]
45594,899080894027362304,drnealdunnfl2,We stopped by the Apalachicola National Estuar...,Republican,[]
45601,899076273355358208,repkinzinger,Nuclear power is vital to our nat'l security. ...,Republican,"[@NEI, @LindseyGrahamSC, @SaxbyChambliss]"


In [ ]:
military_edges = []

for key in tweet_dict:

    if key in military_tweet_df["screen_name"].values:
      ppl = tweet_dict[key]
      for person in ppl:
          military_edges.append((key, person))


G_military = nx.Graph()

for m_edge in military_edges:
    if G_military.has_edge(m_edge[0], m_edge[1]):
        #increase weight
        G_military[m_edge[0]][m_edge[1]]['weight'] += 1
    else:
        #initialize weight
        G_military.add_edge(m_edge[0], m_edge[1], weight=1)

In [ ]:
m_degrees = G_military.degree()

Sigma(G_military, node_size = m_degrees, node_size_scale = ("log"), node_size_range = (0.1, 10), node_color=color_dict)


Sigma(nx.Graph with 975 nodes and 743 edges)

In [ ]:
military_referees =nx.eigenvector_centrality(G_military,max_iter=200)

Sigma(G_military, node_size = military_referees, node_size_scale = ("sqrt"), node_size_range = (1, 10), node_color=color_dict)

Sigma(nx.Graph with 975 nodes and 743 edges)

#  Military Communities

In [ ]:
military_communities = nx.community.louvain_communities(G_military, seed=101)

mc_list = []
for i in military_communities:
  if (len(i) > 5):
    person = list(i)
    mc_list.append(person)

In [ ]:
mc_list

mc_dict = {}

for l in mc_list:
  i = 0
  counter = mc_list.index(l) + 1
  while i < len(l):
    mc_dict[l[i]] = counter
    i += 1



In [ ]:
def search(value):
  if value in mc_dict.keys():
    return mc_dict[value]

military_tweet_df["louvrain"] = military_tweet_df["screen_name"].apply(search)

<ipython-input-38-05b1c02d2fb7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  military_tweet_df["louvrain"] = military_tweet_df["screen_name"].apply(search)


In [ ]:
military_tweet_df.replace("NaN", np.nan, inplace=True)

# Iterating through DataFrame and removing rows with NaN values
for index, row in military_tweet_df.iterrows():
    if row.isnull().any():
        military_tweet_df.drop(index, inplace=True)

Streaming output truncated to the last 5000 lines.
<ipython-input-39-b54ef2fb031a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  military_tweet_df.drop(index, inplace=True)
<ipython-input-39-b54ef2fb031a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  military_tweet_df.drop(index, inplace=True)
<ipython-input-39-b54ef2fb031a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  military_tweet_df.drop(index, inplace=True)
<ipython-inpu

In [ ]:
military_tweet_df

,id,screen_name,text,party,references,louvrain
72,893456317138051072,reppaulmitchell,The health of our #GreatLakes is above politic...,Republican,[@thetimesherald],1.0
167,893470043811532800,repadamsmith,"RAISE Act contradicts American values, places ...",Democrat,[],5.0
197,893465547907760128,replipinski,#ProblemSolvers Plan That Would Prevent 2018 P...,Democrat,[],2.0
239,893487080856109056,bettymccollum04,@NGAUS1878 This award honors Connie’s tireless...,Democrat,[@NGAUS1878],1.0
241,893487036551843840,bettymccollum04,"Connie Haddeland, a Constituent Services Repre...",Democrat,[@NGAUS1878.],1.0
...,...,...,...,...,...,...
45551,899055403627106304,repstephmurphy,A tragic loss for our central Florida communit...,Democrat,"[@OrlandoPolice, @kissimmeepolice]",9.0
45557,899053154670485504,repbillshuster,Last night six police officers were viciously ...,Republican,[],4.0
45575,899065966096560128,senatortester,See amazing speakers @ my first http://lastbes...,Democrat,"[@, @Taldii]",10.0
45601,899076273355358208,repkinzinger,Nuclear power is vital to our nat'l security. ...,Republican,"[@NEI, @LindseyGrahamSC, @SaxbyChambliss]",11.0


In [ ]:
fig = px.histogram(military_tweet_df, x='louvrain', color='party', barmode='group',
            color_discrete_map={'Republican': '#ff0000', 'Democrat': '#0000FF'})

# update the layout of the figure
fig.update_layout(title='Communities on military discussion',
                  xaxis_title='Community',
                  yaxis_title='Political Representation',
                  xaxis={'categoryorder': 'total descending'},
                  height=400, width=900)

# show the plot
fig.show()

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def find_common_words_by_community(df, community_column, text_column, num_words=5):
    result = []

    communities = df[community_column].unique()
    stop_words = set(stopwords.words('english'))
    additional_stop_words = {'http', 'rt', "https", "amp", "qt"}  # Additional words to exclude from common words

    for community in communities:
        # Filter dataframe by community
        community_tweets = df[df[community_column] == community][text_column]

        # Combine tweets into a single text
        combined_text = ' '.join(community_tweets)

        # Tokenize the combined text
        tokens = word_tokenize(combined_text.lower())

        # Filter out stopwords, non-alphabetic tokens, and additional stop words
        filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words and word not in additional_stop_words]

        # Get word frequencies
        word_freq = Counter(filtered_tokens)

        # Get most common words (words only, without frequencies)
        common_words = [word for word, _ in word_freq.most_common(num_words)]

        # Append result as a dictionary
        result.append({'Community': community, 'Common Words': common_words})

    # Create dataframe from the list of dictionaries
    common_words_df = pd.DataFrame(result)
    return common_words_df

# Call the function to get common words by category
common_words_df = find_common_words_by_community(military_tweet_df, 'louvrain', 'text')

# Display the resulting dataframe
common_words_df

,Community,Common Words
0,1.0,"[mnstatefair, realdonaldtrump, thank, great, new]"
1,5.0,"[trump, realdonaldtrump, white, war, potus]"
2,2.0,"[potus, today, veterans, great, must]"
3,4.0,"[trump, realdonaldtrump, utpol, forward, today]"
4,11.0,"[today, gt, must, thanks, north]"
5,7.0,"[fair, job, veterans, kckcctec, today]"
6,17.0,"[safety, veterans, families, repcharliedent, h..."
7,13.0,"[academy, congress, statement, military, days]"
8,3.0,"[thanks, great, see, repfredupton, today]"
9,16.0,"[wyoming, choice, program, improve, fair]"


In [ ]:
pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 16.4 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=5d1a3ea87b27c2f32c098b93d61f6c1e1d33bb86be1b5c77795e69793a8cf23f
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6609c8e72d229529fc6b63222ff79231b6b962e6a632c133f80bacc5e8d92d92
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
summarizer_lsa = LsaSummarizer()


nltk.download('wordnet')

def summarize_tweets_by_community(df, community_column, text_column, num_words=5):
    result = []

    communities = df[community_column].unique()
    stop_words = set(stopwords.words('english'))

    for community in communities:
        # Filter dataframe by community
        community_tweets = df[df[community_column] == community][text_column]

        # Combine tweets into a single text
        combined_text = ' '.join(community_tweets)

        language = "english"

        #Generate Summary
        parser = PlaintextParser.from_string(combined_text, Tokenizer(language))

        summary =summarizer_lsa(parser.document,4)

        print("Summary:", summary)
        # Append result as a dictionary
        result.append({'Community': community, 'Summary': ' '.join(str(summary))})

    # Create dataframe from the list of dictionaries
    summary_df = pd.DataFrame(result)
    return summary_df

summary_df = summarize_tweets_by_community(military_tweet_df, 'louvrain', 'text')

# Merge summary dataframe with common_words_df based on 'Community' column
common_words_df = common_words_df.merge(summary_df, on='Community', how='left')

print("Common words by community with summaries:")
print(common_words_df)

[nltk_data] Downloading package wordnet to /root/nltk_data...


Summary: (<Sentence: https://twitter.com/ABC/status/902976745263030272 QT @ABC NEW: Pentagon announces there are actually 11,000 U.S. troops deployed in Afghanistan, up from the previously-acknowledged number of 8,400.>, <Sentence: Some of WNC's best--Councilwoman Barbara Pennell, Fletcher Mayor Bill Moore, State Senator Ralph Hise, and Yancey Co. Sheriff Garry Banks!>, <Sentence: https://twitter.com/AP/status/902539123608735744 QT @AP In South Sudan, child marriage is rising:  'if you have a mature daughter you give her up to let the rest survive' http://apne.ws/CWRHvnm What an honor to attend the medal ceremony today for WWII D-Day veteran, Herbert Simmons--a tremendous man and hero!>, <Sentence: http://pbs.twimg.com/media/DICO4YKXcAAG9Mt.jpg I wholeheartedly agree with @USATODAY: @realDonaldTrump's climate denial places the future of our planet at risk.>)
Summary: (<Sentence: More info: https://jayapal.house.gov/media/press-releases/jayapal-wilson-introduce-bipartisan-south-asian-he

In [ ]:
environment_phrases = ["paris", "agreement", "climate", "warming", "ocean", "air", "pollution", "heat", "scientific", "nature", "COP", ]

environment_tweet_df = tweets_df[tweets_df['text'].str.contains('|'.join(environment_phrases), case=False)]

environment_tweet_df

,id,screen_name,text,party,references
11,893340028960595968,roslehtinen,RT @CGSydney .@the_AALD is underway! Fascinati...,Republican,[@CGSydney]
24,893440020048224256,reprichhudson,Chris and Jonathan gave their lives protecting...,Republican,[]
25,893439973239771136,reprichhudson,We also pray for those who served alongside th...,Republican,[]
72,893456317138051072,reppaulmitchell,The health of our #GreatLakes is above politic...,Republican,[@thetimesherald]
94,893451740749672448,sendonnelly,Joe visited Pioneer Village at @IndyStateFair ...,Democrat,[@IndyStateFair]
...,...,...,...,...,...
45556,899053223557738496,bettymccollum04,As Heather Heyer's cousin reminds us in this e...,Democrat,[@CNN]
45575,899065966096560128,senatortester,See amazing speakers @ my first http://lastbes...,Democrat,"[@, @Taldii]"
45590,899086914988494848,joaquincastrotx,RT @TexasObserver USDA Rural Development helpe...,Democrat,[@TexasObserver]
45593,899084944772911104,drnealdunnfl2,"This week, I exchanged coins with Chief Master...",Republican,[]


# Health Communities

In [ ]:
health_communities = nx.community.louvain_communities(G_health, seed=101)

hc_list = []
for i in health_communities:
  if (len(i) > 5):
    person = list(i)
    hc_list.append(person)



In [ ]:
hc_dict = {}

for l in hc_list:
  i = 0
  counter = hc_list.index(l) + 1
  while i < len(l):
    hc_dict[l[i]] = counter
    i += 1

def search_h(value):
  if value in hc_dict.keys():
    return hc_dict[value]

health_tweet_df["louvrain"] = health_tweet_df["screen_name"].apply(search_h)


In [ ]:
health_tweet_df.replace("NaN", np.nan, inplace=True)

# Iterating through DataFrame and removing rows with NaN values
for index, row in health_tweet_df.iterrows():
    if row.isnull().any():
        health_tweet_df.drop(index, inplace=True)

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-51-be8c21def674>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-51-be8c21def674>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-51-be8c21def674>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipyt

In [ ]:
fig = px.histogram(health_tweet_df, x='louvrain', color='party', barmode='group',
            color_discrete_map={'Republican': '#ff0000', 'Democrat': '#0000FF'})

# update the layout of the figure
fig.update_layout(title='Communities on health discussion',
                  xaxis_title='Community',
                  yaxis_title='Political Representation',
                  xaxis={'categoryorder': 'total descending'},
                  height=400, width=900)

# show the plot
fig.show()

In [ ]:
health_common_words_df = find_common_words_by_community(health_tweet_df, 'louvrain', 'text')

In [ ]:
health_summary_df = summarize_tweets_by_community(health_tweet_df, 'louvrain', 'text')

# Merge summary dataframe with common_words_df based on 'Community' column
health_common_words_df = health_common_words_df.merge(health_summary_df, on='Community', how='left')

Summary: (<Sentence: @thedayct agrees: Lets build on #ACA to make health care better http://www.theday.com/editorials/20170820/expanding-medicare-is-proposal-worth-debating We need bipartisan solutions for problems in the healthcare system.>, <Sentence: https://twitter.com/news12nj/status/902326439923605504 QT @News12NJ Lawmakers announce changes to Paterson flood maps #NJTonight http://newjersey.news12.com/story/36236570/lawmakers-announce-changes-to-paterson-flood-maps#.WaS0k4ETXwI.twitter Great visit at the Career Academy &amp; Technical School in Troutman.>, <Sentence: Around 50 people turned out to discuss healthcare, crumbling foundations, and education issues http://pbs.twimg.com/media/DH3IztZXsAIMkY6.jpg Caught up with some community leaders in Martin.>, <Sentence: Talked economy, ag, health care, education and pheasant #s. http://pbs.twimg.com/media/DILsVjYUQAAhOxH.jpg AR native #GlenCampbell made our state proud through his legendary music career &amp; brave fight against Alz

In [ ]:
health_common_words_df

,Community,Common Words,Summary
0,3.0,"[care, health, aca, great, bill]",( < S e n t e n c e : @ t h e d a y c t a ...
1,2.0,"[care, health, trump, obamacare, realdonaldtrump]",( < S e n t e n c e : h t t p : / / p b s . ...
2,1.0,"[career, care, obamacare, congrats, legislation]",( < S e n t e n c e : # C h e c k s A n d B ...
3,9.0,"[care, families, flapol, repkevinyoder, work]",( < S e n t e n c e : @ R e p S t e p h M u ...
4,6.0,"[health, care, healthcare, today, us]",( < S e n t e n c e : # P a i d L e a v e ...
5,4.0,"[care, health, leave, people, trump]",( < S e n t e n c e : h t t p s : / / t w i ...
6,7.0,"[healthcare, gt, susanwbrooks, taxreform, new]",( < S e n t e n c e : A b l e 2 t a l k ...
7,5.0,"[town, hall, sensanders, care, need]",( < S e n t e n c e : # I C Y M I : # M I ...
8,8.0,"[health, carson, free, healthcare, senior]",( < S e n t e n c e : @ r e a l D o n a l d ...
9,10.0,"[nihdirector, research, thanks, collins, spend...",( < S e n t e n c e : T h a n k s t o D ...


# Immigration

In [ ]:
border_communities = nx.community.louvain_communities(G_border, seed=101)

bc_list = []
for i in border_communities:
  if (len(i) > 5):
    person = list(i)
    bc_list.append(person)


In [ ]:
bc_dict = {}

for l in bc_list:
  i = 0
  counter = bc_list.index(l) + 1
  while i < len(l):
    bc_dict[l[i]] = counter
    i += 1

def search_b(value):
  if value in bc_dict.keys():
    return bc_dict[value]

border_tweet_df["louvrain"] = border_tweet_df["screen_name"].apply(search_b)


<ipython-input-57-9a2ca8701192>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
border_tweet_df.replace("NaN", np.nan, inplace=True)

# Iterating through DataFrame and removing rows with NaN values
for index, row in border_tweet_df.iterrows():
    if row.isnull().any():
        border_tweet_df.drop(index, inplace=True)

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-58-b97e34223338>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-58-b97e34223338>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-58-b97e34223338>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipyt

In [ ]:
fig = px.histogram(border_tweet_df, x='louvrain', color='party', barmode='group',
            color_discrete_map={'Republican': '#ff0000', 'Democrat': '#0000FF'})

# update the layout of the figure
fig.update_layout(title='Communities on immigration discussion',
                  xaxis_title='Community',
                  yaxis_title='Political Representation',
                  xaxis={'categoryorder': 'total descending'},
                  height=400, width=900)

# show the plot
fig.show()

In [ ]:
border_common_words_df = find_common_words_by_community(border_tweet_df, 'louvrain', 'text')

In [ ]:
border_summary_df = summarize_tweets_by_community(border_tweet_df, 'louvrain', 'text')

# Merge summary dataframe with common_words_df based on 'Community' column
border_common_words_df = border_common_words_df.merge(border_summary_df, on='Community', how='left')

Summary: (<Sentence: https://twitter.com/washingtonpost/status/892809488062918658 QT @washingtonpost Analysis: It’s a "grave mistake" for Trump to cut legal immigration in half http://wapo.st/2vittwP The anti-immigrant RAISE Act goes against our basic values.>, <Sentence: https://www.hatch.senate.gov/public/index.cfm/releases?ID=888889F9-E507-4105-98C1-501B1D2A6948 #115SoFar http://pbs.twimg.com/media/DGuzFIzWsAIl0cC.jpg Meet @USArmy Cadet Simone Askew, from Fairfax County, currently breaking barriers at @WestPoint_USMA:  https://www.nytimes.com/2017/08/14/us/west-point-black-female-first.html?mcubz=1 Trump would rather talk about his border wall, tax cuts, political attacks, etc today than condemn white supremacists for domestic terrorism http://pbs.twimg.com/media/DHMTHNHXoAAhOqO.jpg - failed to investigate sex crimes against immigrant children - referred to part of his prison "a concentration camp" - pushed birtherism 2/ @realDonaldTrump It's no coincidence that hate crimes spiked w

In [ ]:
border_common_words_df

,Community,Common Words,Summary
0,2.0,"[trump, wall, immigration, immigrants, would]",( < S e n t e n c e : h t t p s : / / t w i ...
1,1.0,"[immigration, raise, act, immigrants, working]",( < S e n t e n c e : W A T C H : h t t p ...
2,8.0,"[border, working, hate, new, consent]",( < S e n t e n c e : h t t p s : / / t w i ...
3,11.0,"[crime, great, trump, hate, yet]",( < S e n t e n c e : R e a d m y o p - ...
4,7.0,"[court, act, potus, us, immigration]",( < S e n t e n c e : # D e f e n d D A C A ...
5,4.0,"[immigration, border, housejuddems, national, ...",( < S e n t e n c e : h t t p s : / / t w i ...
6,10.0,"[border, mississippi, policy, conference, black]",( < S e n t e n c e : T h r e e m o r e ...
7,3.0,"[immigration, immigrants, trump, realdonaldtru...",( < S e n t e n c e : W a s h o n o r e d ...
8,9.0,"[border, potus, great, borders, westerncaucus]",( < S e n t e n c e : @ R e p N e w h o u s ...
9,5.0,"[american, drug, dream, fight, discuss]",( < S e n t e n c e : S l a s h i n g l e ...
